### Data Mining

*"We are drowning in data, but starving for knowledge"*

- Data Mining is the process of automated analysis of large data sets.
- Data Mining is the extraction of interesting ( non-trivial, previously unknown, implicit and potential useful) patterns or knowledge from huge amount of data.
- Data Mining is also known as Business Intelligence (BI), Knowledge Discovery in DAtabases (KDD), Data Analysis etc.



Multi Dimensional Data Cube

Data are usually stored as multi-dimensional cubes. The data is stored as a summerized version like count, avg, sum, etc. rather than showing specifics of transactions. Each cell of the cube represents some kind of summerized data

- Drill Down: Ability to get into minute details. like Quarters can be further divided into months
- Roll up: Ability to further summerized. like cities can be combined together as countries.





#### Data Cube

- Data cube is a latice of cuboids
- N dim data cube is a cube with N dimensions or attributes.
- Measures are represented in the cells.
- Base cell: Measures in the base cuboid are called base cells
- Aggregate cell: Any measure in the non-base lattice is called aggregate cell. And an aggregate cell aggregates over one or more dimensions.
- For a n-dim cube, cells are represented as m-dimensional cells. if m==n, then the cell is base cell else its an aggregate cell
- Ancestor and descendant cells - A cell a of i-dim is called an ancestor cell b of j-dim if and only if i < j.

#### Data Cube Materialization

Pre-computing the cuboids of a data cube and storing it in the file system ahead of time, so that at the time of data-analysis, the process of querying become faster is called Data Cube Materialization

- Full Cube Materialization
- Partial Cube Materialization
   - Iceberg Cubes:  Most of the time, a substantial amount of cube's space is taken up by measures with very low values. This is because, the cubes are mostly sparsely distributed within a multidimensional space. In such cases, it is sometimes useful to materialize only those cells in the cuboid whose values are above a certain threshold. Such, partially materialized cubes are known as ice-berg cubes.
   The minimum threshold is called the **Minimum Support Threshold**. The constraint specified in the having clause is called as **iceberg condition**

Problems with Data Cube pre computation
- huge computational power
- huge storage space

Efficient methods for data cube computation
- Multiway
    - It computes full data cube by using a multidim array as its basic structure.
    - It uses direct array addressing, where dimension values are accesses via a position or index of their corresponding array location
    - Hence, its difficult to perform any value-based reordering as an optimization technique.
    - Solution- to partition the array into chunks. A **chunk** is a subcube that is small enough to fit into the memory available for cube computation
    - ChunkID+offset can be used as a cell-addressing mechanism and also to compress sparse arrays
    - This chunking technique involves “overlapping” some of the aggregation computations; therefore, it is referred to as multiway array aggregation. It performs simultaneous aggregation, that is, it computes aggregations simultaneously on multiple dimensions.
    - Some of the aggregation is done in parallel. For e.g. when a 3d chuck is being scanned, all its associated 2d chuckes are computed. For e.g. when $a_0b_0c_0$ is being scanned, 2d chunks $a_0b_0$, $b_0c_0$, $a_0c_0$ are computed
    - MultiWay is most effective when the product of the cardinalities of dimensions is moderate and the data are not too sparse. When the dimensionality is high or the data are very sparse, the in-memory arrays become too large to fit in memory, and this method becomes infeasible.
    - MultiWay is not good for computing Iceberg cube since multiway goes bottom up while Apiori property states that if a cell doesn't satisfy the minimum support, neither will its descendant do and hence iceberg goes top down which cannot be achieved using MultiWay

- BUC (Bottom-up construction)
    - BUC is an algorithm for the computation of sparse and iceberg cubes.
    - 
- Star Cubing
    - It integrates top-down and bottom-up cube computation and explores both multi dimensional aggregation(similar to Multiway) and Apriori like pruning(similar to BUC) 
    - It operates from a data structure called a **Star Tree** which performs lossless data compression, there by reducing the computation time and memory requirements
    - **Shared dimensions**: 

- Shell fragment



In [1]:
import xmltodict

### Post Edited Text

In [ ]:
with open('PE_EN-PT.xml',encoding='utf-8') as fd:
    doc = xmltodict.parse(fd.read(),encoding='utf-8')

In [ ]:
records = doc['QualitivityProfessional']['Client']['Project']['Activity']['Document']['Record']

In [ ]:
dict(records[0]['contentText'])['source']

In [ ]:
keystrokes = dict(records[0]['keyStrokes'])['ks']

In [ ]:
keystrokes

In [ ]:
def extractTargetText(xmlDoc):
    records = xmlDoc['QualitivityProfessional']['Client']['Project']['Activity']['Document']['Record']
    for record in records:
        source = record['contentText']['source']
        targetOriginal = record['contentText']['targetOriginal']
        targetUpdated = record['contentText']['targetUpdated']
        keystrokes = record['keyStrokes']['ks']
        #print(keystrokes)
        print(targetUpdated,end='\n\n')
        constructTargetText(keystrokes)
        break
        
    
def constructTargetText(keystrokes):
    special_keys = ['[Back]','[Space]']
    for ks in keystrokes:
        system = ks.get('@system')
        if system:
            
            global original_text 
            original_text = ks.get('@text')
            break
    print(f"Original Text = {original_text}")
    
    orig_text = [w for w in original_text]
    #print(''.join(orig_text))
    print(len(orig_text))
    for ks in keystrokes:
        text = ks.get('@text')
        key = ks.get('@key')
        position = ks.get('@position')
        pos = int(position)
        created = ks.get('@created')
        selection = ks.get('@selection')
        system = ks.get('@system')
        if system:
            continue
        print(f"At {position}  character ---> '{orig_text[pos]}' ---> text: '{text}' ----> key pressed: '{key}' ---> selection: '{selection}'")
        if selection:
            temp = handle_selection(orig_text, selection, pos, text, key)
            orig_text = temp
        elif key == '[BACK]':
            print("inside [BACK]")
            #pass
        else:
            print("Inside else")
            if len(text) > 1:
                for i,c in enumerate(text):
                    orig_text.insert(pos+i,c)
            else:
                orig_text.insert(pos,text)
            print(''.join(orig_text))
        
        
def handle_selection(temp, selection, position, text, key):
    print("inside handle_selection")
   # position = int(position)
    #print(''.join(temp))
    start = position - len(selection)
    #print(start)
    remove_index = [i for i in range(start,position)]
    print(f"index to remove----> {remove_index}")
    #delete the characters
    print(f"'{''.join(temp[start:position])}'")
    print(selection)
    del(temp[start:position])
    #start = start+1
    if key == '[Space]':
        temp.insert(start,' ')
    #elif key == '[Back]':
        # Do nothing as its already deleted above
        #pass
    else:
        #print(f"length of text = {len(text)}")
        # if text field is non empty
        text = text.rstrip()
        if text:
            for i,c in enumerate(text):
                temp.insert(start+i,c)
           
    print(''.join(temp))
    
    return temp

In [ ]:
extractTargetText(doc)

In [ ]:
x = 'axx '

In [ ]:
x.rstrip()

In [316]:
x = []

In [317]:
x.insert(1,1)

In [318]:
x

[1]

In [ ]:
Críticas crescem sobre o fatrdoehnson patrerohnson para discursi scursos políticos
Críticas crescem sobre o fato de Johnson ter usado a polícia por para discursos políticos

### Translated Text

In [ ]:
with open('Translation_PT-EN.xml',encoding='utf-8') as fd:
    translated_doc = xmltodict.parse(fd.read(),encoding='utf-8')

- while inserting, it starts inserting at index = 1
- that means if we enter [how], h=1, o=2, w=3, but it actually gets stored at h=0, w=1, w=2
- while deleting, it deletes at the actual index
- suppose it wants to delete w, it will do so at index=2, rather than 3

In [247]:
def extractTargetText1(xmlDoc,record_number=5):
    records = xmlDoc['QualitivityProfessional']['Client']['Project']['Activity']['Document']['Record']
    #for record in records:
    record = records[record_number]
    source = record['contentText']['source']
    targetOriginal = record['contentText']['targetOriginal']
    targetUpdated = record['contentText']['targetUpdated']
    keystrokes = record['keyStrokes']['ks']
    #print(keystrokes)
    print(targetUpdated,end='\n\n')
    extractedText = constructTargetText1(keystrokes, targetOriginal)
    return extractedText,targetUpdated
        
    
def constructTargetText1(keystrokes, targetOriginal):
    special_keys = ['[Back]','[Space]']
    
    if targetOriginal:
        orig_text = [i for i in targetOriginal]
    else:
        orig_text = []
    
    print(len(orig_text))
   
    for ks in keystrokes:
        text = ks.get('@text')
        key = ks.get('@key')
        position = ks.get('@position')
        pos = int(position)
        created = ks.get('@created')
        selection = ks.get('@selection')
        system = ks.get('@system')
        if orig_text:
            print(f"\n[DEBUG] At {position}  ---> text: '{text}' ----> key pressed: '{key}' ---> selection: '{selection}'")
        if selection:
            temp = handle_selection1(orig_text, selection, pos, text, key)
            orig_text = temp
        elif key == '[BACK]':
            print("[DEBUG] Delete text")
            #pass
        else:
            print("[DEBUG] Insert Text")
            if len(text) > 1:
                for i,c in enumerate(text):
                    orig_text.insert(pos+i,c)
            else:
                orig_text.insert(pos,text)
            updated_text = ''.join(orig_text)
            print(f"'{updated_text}'")
    return orig_text
        
def handle_selection1(temp, selection, position, text, key):
    print("[DEBUG] Select and Modify text")
   # position = int(position)
    #print(''.join(temp))
    if len(selection) == 1:
        #if key == '[Back]':
        start = position - len(selection)+1
        end = position+1
        remove_index = [i for i in range(start,end)]
        print(f"[DEBUG] index to remove----> {remove_index}")
        #delete the characters
        to_delete = ''.join(temp[start:end])
        print(f"[DEBUG] To delete: '{to_delete}'")
        print(f"[DEBUG] Selection: '{selection}'")
        if to_delete != selection:
            print(f"[ERROR] Selection string doesn't match to_delete string!!!")
        del(temp[start:end])
        #else:
            
    elif len(selection) > 1:
        if key == '[Back]':
            start = position - 1 + 1
            end = start+len(selection)+1
        else:
              # creates problem in record 5
              
            start = position - 1
            # +1 deletes an extra
            end = start+len(selection)
        remove_index = [i for i in range(start, end)]
        print(f"[DEBUG] index to remove----> {remove_index}")
        #delete the characters
        to_delete = ''.join(temp[start:end])
        print(f"[DEBUG] To delete multiple: '{to_delete}'")
        print(f"[DEBUG] Multiple Selection: '{selection}'")
        if to_delete != selection:
              print(f"[ERROR] Selection string doesn't match to_delete string!!!")
        del(temp[start:end])
    #start = start+1
    if key == '[Space]':
        temp.insert(start,' ')
    #elif key == '[Back]':
        # Do nothing as its already deleted above
        #pass
    else:
        #print(f"length of text = {len(text)}")
        # if text field is non empty
        #text = text.rstrip()
        if text:
            for i,c in enumerate(text):
                temp.insert(start+i,c)
           
    print(''.join(temp))
    
    return temp


In [141]:
print(targetUpdated)
print(''.join(extractedText))

The effects of the hurricane have already been felt in the South East of the United States, with gales and torrential rain on Thursday (5) after the hurricane had already devastated the north of the Bahamas islands, where it left 30 dead and thousands homeless.
The effects of the hurricane have already been felt in the SoutheE st of the United States, with gales and torrential rain on Thursday (5) after the hurricane had already devastated the north of the Bahamas islands, where it left 30 dead and thousands homeless.


In [110]:
records = translated_doc['QualitivityProfessional']['Client']['Project']['Activity']['Document']['Record']

In [313]:
aa = [0,1,2,3,4,5]

In [314]:
aa.insert(3,55)

In [315]:
aa

[0, 1, 2, 55, 3, 4, 5]

In [285]:
del(aa[1:3])

In [286]:
aa

[0, 3, 4, 5]

In [288]:
[i for i in range(1,3)]

[1, 2]

In [311]:
import datetime
from dateutil.parser import parse

def captureKeystrokes(source_xml, record_number=2, debug=False):
    # To store the keystores for Translog
    captured_keystrokes = []
    # To store the previous timestamp of each keystroke
    first_ts = 0.0
    
    records = source_xml['QualitivityProfessional']['Client']['Project']['Activity']['Document']['Record']
    #for record in records:
    record = records[record_number]
    source = record['contentText']['source']
    targetOriginal = record['contentText']['targetOriginal']
    targetUpdated = record['contentText']['targetUpdated']
    keystrokes = record['keyStrokes']['ks']
    
    # orig_text keeps the track of the reconstructed text so far
    if targetOriginal:
        orig_text = [i for i in targetOriginal]
    else:
        orig_text = []
    
    for count, ks in enumerate(keystrokes):
        text = ks.get('@text')
        key = ks.get('@key')
        position = ks.get('@position')
        pos = int(position)
        created = ks.get('@created')
        selection = ks.get('@selection')
        system = ks.get('@system')
        
        tt = parse(created)
        tt = tt.timestamp()
        if count == 0:
            first_ts = tt
            tt = 0
        else:
            tt = tt - first_ts
        # convert tt into miliseconds
        tt = int(tt*1000)
        
        if selection:
            print("[DEBUG] Handling selection(first delete and then insert) characters")
            orig_text, ks_list = extractSelectionKeystrokes(orig_text, pos, text, key, tt, selection)
            #curr_updated_text = ''.join(orig_text)
            #print(f"[DEBUG] Current Text = '{curr_updated_text}'")
            for ks in ks_list:
                captured_keystrokes.append(ks)
        else:
            opType = ''
            if key == '[Back]':
                del(orig_text[pos])
                curr_updated_text = ''.join(orig_text)
                if debug:
                    print("[DEBUG] Deleting characters")
                    print(f"[DEBUG] Current Text = '{curr_updated_text}'")
                opType = "delete"
            else:
                for index,char in enumerate(text):
                    orig_text.insert(pos+index, char)
                curr_updated_text = ''.join(orig_text)
                if debug:
                    print("[DEBUG] Inserting characters")
                    print(f"[DEBUG] Current Text = '{curr_updated_text}'")
                
                opType = "insert"
        
            target_ks = {'time': str(tt), 'cursor': position, 'type': opType, 'value': text}
            captured_keystrokes.append(target_ks)
    print(f"[INFO] Original Text: {''.join(orig_text)}")
        
    return source, targetUpdated, captured_keystrokes
    
def extractSelectionKeystrokes(orig_text, position, text, key, time, selection):
    curr_updated_text = ''.join(orig_text)
    print(f"[DEBUG] Current Text before updating = '{curr_updated_text}'")
    ks_list = []
    if(len(selection) == 1):
        start = position
        print(f"\t[DEBUG]Single: key = {key} : character at position: {start} is '{orig_text[start]}', selection = '{selection}'")
        if selection == orig_text[start]:
            print(f"\t\t[DEBUG] Deleting '{orig_text[start]}'")
            del(orig_text[start])
        elif selection == orig_text[start-1]:
            print(f"\t\t[DEBUG] Deleting '{orig_text[start-1]}'")
            del(orig_text[start-1])
        else:
            print("[ERROR] selection not found")
    else:
        
        if key == '[Back]':
            start = position
            end = start + len(selection) + 1
        else:
            start = position - 1
            # +1 deletes an extra
            end = start + len(selection)
        print(f"\t[DEBUG]Multiple: key = {key} : characters around position {position}({orig_text[position]}): '{''.join(orig_text[position-2:position+1])}'")    
        remove_index = [i for i in range(start, end)]
        print(f"[DEBUG] index to remove: {remove_index}")
        #delete the characters
        to_delete = ''.join(orig_text[start:end])
        print(f"[DEBUG] To delete multiple: '{to_delete}'")
        print(f"[DEBUG] Multiple Selection: '{selection}'")
        if to_delete != selection:
              print(f"[ERROR] Selection string doesn't match to_delete string!!!")
        del(orig_text[start:end])
    
    target_ks = {'time': str(time), 'cursor': start, 'type': "delete", 'value': selection}
    ks_list.append(target_ks)
    
    # Insert the characters in the text field
    
    if key == '[Space]':
        orig_text.insert(start,' ')
        target_ks = {'time': str(time), 'cursor': start, 'type': "insert", 'value': ' '}
        ks_list.append(target_ks)
    else:
        
        if text:
            for i,c in enumerate(text):
                orig_text.insert(start+i,c)
            target_ks = {'time': str(time), 'cursor': start, 'type': "insert", 'value': text}
            ks_list.append(target_ks)

    return orig_text, ks_list


def addKeystrokes(keystrokes, target_xml, debug=False):
    if not isinstance(target_xml, OrderedDict):
        print("[ERROR] Enter a valid xml file")
        return
    if not keystrokes:
        print("[ERROR] No keystores to add")
        return
    if target_xml.get('LogFile').get('Events'):
        target_xml['LogFile']['Events']['Key'] = []
    else:
        target_xml['LogFile']['Events'] = OrderedDict()
        target_xml['LogFile']['Events']['Key'] = []
        
    keys = target_xml['LogFile']['Events']['Key']
    for ks in keystrokes:
        keys.append(addKsToDict(ks))
    
    target_xml['LogFile']['Events']['Key'] = keys
    
    return target_xml

def addSourceText(sourceText, target_xml):
    target_xml['LogFile']['Project']['Interface']['Standard']['Settings']['SourceText'] = sourceText
    return target_xml

def addTargetText(targetText, target_xml):
    target_xml['LogFile']['Project']['Interface']['Standard']['Settings']['TargetText'] = targetText
    return target_xml
    
from collections import OrderedDict

def addKsToDict(keystrokes_dic):
    new_dict = {'@Time': keystrokes_dic.get('time'), '@Cursor': keystrokes_dic.get('cursor'), '@Type': keystrokes_dic.get('type'), '@Value': keystrokes_dic.get('value')}
    return OrderedDict(new_dict)

### Capture keystrokes from Qualitifity source xml file

In [312]:
extractedText, targetUpdated, captured_keystrokes = captureKeystrokes(translated_doc,4)

[DEBUG] Handling selection(first delete and then insert) characters
[DEBUG] Current Text before updating = 'The effects of the hurricane have be'
	[DEBUG]Single: key = [Back] : character at position: 35 is 'e', selection = 'e'
		[DEBUG] Deleting 'e'
[DEBUG] Handling selection(first delete and then insert) characters
[DEBUG] Current Text before updating = 'The effects of the hurricane have b'
	[DEBUG]Single: key = [Back] : character at position: 34 is 'b', selection = 'b'
		[DEBUG] Deleting 'b'
[DEBUG] Handling selection(first delete and then insert) characters
[DEBUG] Current Text before updating = 'The effects of the hurricane have alreasdy b'
	[DEBUG]Single: key = [Back] : character at position: 43 is 'b', selection = 'b'
		[DEBUG] Deleting 'b'
[DEBUG] Handling selection(first delete and then insert) characters
[DEBUG] Current Text before updating = 'The effects of the hurricane have alreasdy '
	[DEBUG]Single: key = [Back] : character at position: 42 is ' ', selection = ' '
		[DEBUG]

### Open a Translog xml template file

In [268]:
with open('translog_template.xml',encoding='utf-8') as fd:
    target_xml = xmltodict.parse(fd.read(),encoding='utf-8')

In [269]:
updated_xml = addKeystrokes(captured_keystrokes, target_xml)
#updated_xml = addSourceText(extractedText, target_xml)
updated_xml = addTargetText(targetUpdated, target_xml)

In [270]:
new_xml = xmltodict.unparse(updated_xml)

### Save target File

In [271]:
f = open('translog_template_new.xml','w')
f.write(new_xml)
f.close()

In [234]:


def createOrderedDict(time='',k_type='',value=''):
    keystrokes_dic = {'@Time': time, '@Type': k_type, '@Value': value}
    return OrderedDict(keystrokes_dic)
    

In [164]:
keys

[OrderedDict([('@Time', '53250'),
              ('@Cursor', '0'),
              ('@X', '3'),
              ('@Y', '510'),
              ('@Width', '20'),
              ('@Height', '30'),
              ('@Type', 'insert'),
              ('@Value', 'S')]),
 OrderedDict([('@Time', '53859'),
              ('@Cursor', '1'),
              ('@X', '16'),
              ('@Y', '510'),
              ('@Width', '20'),
              ('@Height', '30'),
              ('@Type', 'insert'),
              ('@Value', 'p')]),
 OrderedDict([('@Time', '719281'),
              ('@Cursor', '875'),
              ('@X', '927'),
              ('@Y', '804'),
              ('@Width', '20'),
              ('@Height', '30'),
              ('@Type', 'insert'),
              ('@Value', 'o')]),
 OrderedDict([('@Time', '719547'),
              ('@Cursor', '876'),
              ('@X', '939'),
              ('@Y', '804'),
              ('@Width', '14'),
              ('@Height', '30'),
              ('@Type', 'insert'),
  